<a href="https://colab.research.google.com/github/LaFuego20/exchange-rate-forecasting/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Single-cell end-to-end LSTM pipeline (Colab / Jupyter)
# Load -> Prepare sequences -> Train 3 LSTMs (Full / Pre / Post) -> Evaluate (normalized + denorm) -> Plot & save results

import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# -------------------------
# 0. Reproducibility
# -------------------------
os.environ['PYTHONHASHSEED'] = '2'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(2)
np.random.seed(2)
tf.random.set_seed(2)

# -------------------------
# 1. PARAMETERS (edit if needed)
# -------------------------
N_TIMESTEPS = 10                # lookback window used in your snippets
BATCH_SIZE = 32
EPOCHS = 100
PATIENCE = 10
SUBSIDY_REMOVAL_DATE = pd.to_datetime('2023-05-29', format='%Y-%m-%d')

# Paths - change if your files are elsewhere
NORMALIZED_CSV = "/content/normalized_data.csv"       # contains USD_NGN_Norm, EUR_NGN_Norm, GBP_NGN_Norm and Date
ORIGINAL_CSV = "/content/merged_exchange_rates.csv"   # contains USD_NGN, EUR_NGN, GBP_NGN and Date

# Target column names (original and normalized)
original_cols = ['USD_NGN', 'EUR_NGN', 'GBP_NGN']
normalized_cols = ['USD_NGN_Norm', 'EUR_NGN_Norm', 'GBP_NGN_Norm']
currencies = ['USD', 'EUR', 'GBP']

# -------------------------
# 2. Load normalized dataset (for training)
# -------------------------
df_norm = None
try:
    df_norm = pd.read_csv(NORMALIZED_CSV, parse_dates=['Date'])
    df_norm.set_index('Date', inplace=True)
    print(f"Loaded normalized data: {NORMALIZED_CSV} (rows: {len(df_norm)})")
except FileNotFoundError:
    print(f"Error: {NORMALIZED_CSV} not found. Please upload or adjust the path.")
except Exception as e:
    print("Error loading normalized data:", e)

# Validate normalized columns
if df_norm is None:
    raise SystemExit("Normalized dataframe not loaded. Fix file path and re-run.")
missing_norm = [c for c in normalized_cols if c not in df_norm.columns]
if missing_norm:
    raise SystemExit(f"Missing normalized columns: {missing_norm}. Ensure your CSV has these columns.")

# Keep only normalized columns used for modeling
df_normalized = df_norm[normalized_cols].copy()

# Ensure datetime index
if not isinstance(df_normalized.index, pd.DatetimeIndex):
    df_normalized.index = pd.to_datetime(df_normalized.index, errors='coerce')
    df_normalized.dropna(axis=0, inplace=True)

# -------------------------
# 3. Helper: create sequences
# -------------------------
def create_sequences(data: np.ndarray, n_steps: int):
    X, y = [], []
    if len(data) <= n_steps:
        return np.array(X), np.array(y)
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps])
        y.append(data[i + n_steps])
    return np.array(X), np.array(y)

# -------------------------
# 4. Split into periods
# -------------------------
df_full = df_normalized.dropna().copy()
df_pre = df_normalized[df_normalized.index < SUBSIDY_REMOVAL_DATE].dropna().copy()
df_post = df_normalized[df_normalized.index >= SUBSIDY_REMOVAL_DATE].dropna().copy()

print(f"Full rows: {len(df_full)}, Pre rows: {len(df_pre)}, Post rows: {len(df_post)}")

# -------------------------
# 5. Create sequences and train/test splits
# -------------------------
# Full
X_full, y_full = create_sequences(df_full.values, N_TIMESTEPS)
if X_full.size:
    X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
        X_full, y_full, test_size=0.2, shuffle=False, random_state=2)
else:
    X_train_full = X_test_full = y_train_full = y_test_full = np.array([])

# Pre
X_pre, y_pre = create_sequences(df_pre.values, N_TIMESTEPS)
if X_pre.size:
    X_train_pre, X_test_pre, y_train_pre, y_test_pre = train_test_split(
        X_pre, y_pre, test_size=0.2, shuffle=False, random_state=2)
else:
    X_train_pre = X_test_pre = y_train_pre = y_test_pre = np.array([])

# Post
X_post, y_post = create_sequences(df_post.values, N_TIMESTEPS)
if X_post.size:
    X_train_post, X_test_post, y_train_post, y_test_post = train_test_split(
        X_post, y_post, test_size=0.2, shuffle=False, random_state=2)
else:
    X_train_post = X_test_post = y_train_post = y_test_post = np.array([])

# Print shapes for verification
def print_shapes(label, Xtr, Xte, ytr, yte):
    if isinstance(Xtr, np.ndarray) and Xtr.size:
        print(f"{label} -> X_train: {Xtr.shape}, X_test: {Xte.shape}, y_train: {ytr.shape}, y_test: {yte.shape}")
    else:
        print(f"{label} -> Not enough data for sequences (skipped).")

print_shapes('Full', X_train_full, X_test_full, y_train_full, y_test_full)
print_shapes('Pre', X_train_pre, X_test_pre, y_train_pre, y_test_pre)
print_shapes('Post', X_train_post, X_test_post, y_train_post, y_test_post)

# -------------------------
# 6. Model builder helper
# -------------------------
def build_lstm_model(n_timesteps, n_features, n_outputs):
    model = Sequential([
        LSTM(64, activation='relu', input_shape=(n_timesteps, n_features)),
        Dense(32, activation='relu'),
        Dense(n_outputs)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

early_stop = EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)

# -------------------------
# 7. Train models (Full, Pre, Post)
# -------------------------
model_full = None
if isinstance(X_train_full, np.ndarray) and X_train_full.size:
    model_full = build_lstm_model(N_TIMESTEPS, X_train_full.shape[2], y_train_full.shape[1])
    print("Full model summary:")
    model_full.summary()
    if isinstance(X_test_full, np.ndarray) and X_test_full.size:
        model_full.fit(X_train_full, y_train_full, validation_data=(X_test_full, y_test_full),
                       epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stop], verbose=0)
    else:
        model_full.fit(X_train_full, y_train_full, epochs=EPOCHS, batch_size=BATCH_SIZE,
                       callbacks=[early_stop], verbose=0)
    print("Trained Full model.")

model_pre = None
if isinstance(X_train_pre, np.ndarray) and X_train_pre.size:
    model_pre = build_lstm_model(N_TIMESTEPS, X_train_pre.shape[2], y_train_pre.shape[1])
    print("Pre model summary:")
    model_pre.summary()
    if isinstance(X_test_pre, np.ndarray) and X_test_pre.size:
        model_pre.fit(X_train_pre, y_train_pre, validation_data=(X_test_pre, y_test_pre),
                      epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stop], verbose=0)
    else:
        model_pre.fit(X_train_pre, y_train_pre, epochs=EPOCHS, batch_size=BATCH_SIZE,
                      callbacks=[early_stop], verbose=0)
    print("Trained Pre-subsidy model.")

model_post = None
if isinstance(X_train_post, np.ndarray) and X_train_post.size:
    model_post = build_lstm_model(N_TIMESTEPS, X_train_post.shape[2], y_train_post.shape[1])
    print("Post model summary:")
    model_post.summary()
    if isinstance(X_test_post, np.ndarray) and X_test_post.size:
        model_post.fit(X_train_post, y_train_post, validation_data=(X_test_post, y_test_post),
                       epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stop], verbose=0)
    else:
        model_post.fit(X_train_post, y_train_post, epochs=EPOCHS, batch_size=BATCH_SIZE,
                       callbacks=[early_stop], verbose=0)
    print("Trained Post-subsidy model.")

# -------------------------
# 8. Predictions & Normalized Metrics
# -------------------------
metrics_summary = {}

# Full
y_pred_full = np.array([])
if model_full is not None and isinstance(X_test_full, np.ndarray) and X_test_full.size:
    y_pred_full = model_full.predict(X_test_full, verbose=0)
    rmse_full = np.sqrt(mean_squared_error(y_test_full, y_pred_full))
    mae_full = mean_absolute_error(y_test_full, y_pred_full)
    rmse_full_currency = [np.sqrt(mean_squared_error(y_test_full[:, i], y_pred_full[:, i])) for i in range(len(currencies))]
    mae_full_currency = [mean_absolute_error(y_test_full[:, i], y_pred_full[:, i]) for i in range(len(currencies))]
    metrics_summary['Full'] = {'RMSE_Overall': rmse_full, 'MAE_Overall': mae_full,
                               'RMSE_Currency': rmse_full_currency, 'MAE_Currency': mae_full_currency}
else:
    metrics_summary['Full'] = {'RMSE_Overall': None, 'MAE_Overall': None, 'RMSE_Currency': [], 'MAE_Currency': []}

# Pre
y_pred_pre = np.array([])
if model_pre is not None and isinstance(X_test_pre, np.ndarray) and X_test_pre.size:
    y_pred_pre = model_pre.predict(X_test_pre, verbose=0)
    rmse_pre = np.sqrt(mean_squared_error(y_test_pre, y_pred_pre))
    mae_pre = mean_absolute_error(y_test_pre, y_pred_pre)
    rmse_pre_currency = [np.sqrt(mean_squared_error(y_test_pre[:, i], y_pred_pre[:, i])) for i in range(len(currencies))]
    mae_pre_currency = [mean_absolute_error(y_test_pre[:, i], y_pred_pre[:, i]) for i in range(len(currencies))]
    metrics_summary['Pre-Subsidy'] = {'RMSE_Overall': rmse_pre, 'MAE_Overall': mae_pre,
                                      'RMSE_Currency': rmse_pre_currency, 'MAE_Currency': mae_pre_currency}
else:
    metrics_summary['Pre-Subsidy'] = {'RMSE_Overall': None, 'MAE_Overall': None, 'RMSE_Currency': [], 'MAE_Currency': []}

# Post
y_pred_post = np.array([])
if model_post is not None and isinstance(X_test_post, np.ndarray) and X_test_post.size:
    y_pred_post = model_post.predict(X_test_post, verbose=0)
    rmse_post = np.sqrt(mean_squared_error(y_test_post, y_pred_post))
    mae_post = mean_absolute_error(y_test_post, y_pred_post)
    rmse_post_currency = [np.sqrt(mean_squared_error(y_test_post[:, i], y_pred_post[:, i])) for i in range(len(currencies))]
    mae_post_currency = [mean_absolute_error(y_test_post[:, i], y_pred_post[:, i]) for i in range(len(currencies))]
    metrics_summary['Post-Subsidy'] = {'RMSE_Overall': rmse_post, 'MAE_Overall': mae_post,
                                       'RMSE_Currency': rmse_post_currency, 'MAE_Currency': mae_post_currency}
else:
    metrics_summary['Post-Subsidy'] = {'RMSE_Overall': None, 'MAE_Overall': None, 'RMSE_Currency': [], 'MAE_Currency': []}

print("\n--- Normalized metrics (from normalized data) ---")
display(metrics_summary)

# Save normalized metrics to CSV-friendly dataframe (optional)
try:
    norm_df_rows = []
    for period, d in metrics_summary.items():
        row = {
            'Period': period,
            'RMSE_Overall': d['RMSE_Overall'],
            'MAE_Overall': d['MAE_Overall']
        }
        for i, c in enumerate(currencies):
            row[f'RMSE_{c}'] = d['RMSE_Currency'][i] if d['RMSE_Currency'] else None
            row[f'MAE_{c}'] = d['MAE_Currency'][i] if d['MAE_Currency'] else None
        norm_df_rows.append(row)
    norm_metrics_df = pd.DataFrame(norm_df_rows)
    norm_metrics_df.to_csv('lstm_normalized_metrics_summary.csv', index=False)
    print("Saved lstm_normalized_metrics_summary.csv")
except Exception as e:
    print("Could not save normalized metrics CSV:", e)

# -------------------------
# 9. Denormalize predictions & test sets (min-max)
# -------------------------
df_orig = None
try:
    df_orig = pd.read_csv(ORIGINAL_CSV, parse_dates=['Date'], dayfirst=True)
    df_orig.set_index('Date', inplace=True)
    print(f"Loaded original data: {ORIGINAL_CSV} (rows: {len(df_orig)})")
except FileNotFoundError:
    print(f"Warning: {ORIGINAL_CSV} not found. Denormalization skipped.")
except Exception as e:
    print("Warning: error loading original CSV, denormalization may fail:", e)

min_vals = None
max_vals = None
if df_orig is not None:
    missing_orig = [c for c in original_cols if c not in df_orig.columns]
    if missing_orig:
        print("Warning: original file missing columns:", missing_orig)
    else:
        min_vals = df_orig[original_cols].min()
        max_vals = df_orig[original_cols].max()

# Denormalize helper (expects arrays shape (samples, 3))
def denormalize_array(arr_norm, min_vals, max_vals):
    if arr_norm is None or arr_norm.size == 0:
        return np.array([])
    scale = (max_vals.values - min_vals.values)
    return arr_norm * scale + min_vals.values

y_test_full_denorm = denormalize_array(y_test_full, min_vals, max_vals) if min_vals is not None else np.array([])
y_pred_full_denorm = denormalize_array(y_pred_full, min_vals, max_vals) if min_vals is not None else np.array([])

y_test_pre_denorm = denormalize_array(y_test_pre, min_vals, max_vals) if min_vals is not None else np.array([])
y_pred_pre_denorm = denormalize_array(y_pred_pre, min_vals, max_vals) if min_vals is not None else np.array([])

y_test_post_denorm = denormalize_array(y_test_post, min_vals, max_vals) if min_vals is not None else np.array([])
y_pred_post_denorm = denormalize_array(y_pred_post, min_vals, max_vals) if min_vals is not None else np.array([])

# Recalculate denormalized metrics
rmse_full_currency_denorm = []
mae_full_currency_denorm = []
rmse_pre_list_denorm = []
mae_pre_list_denorm = []
rmse_post_list_denorm = []
mae_post_list_denorm = []

print("\n--- Denormalized metrics (if original CSV was provided) ---")
if y_test_full_denorm.size:
    for i, name in enumerate(currencies):
        rmse = np.sqrt(mean_squared_error(y_test_full_denorm[:, i], y_pred_full_denorm[:, i]))
        mae = mean_absolute_error(y_test_full_denorm[:, i], y_pred_full_denorm[:, i])
        rmse_full_currency_denorm.append(rmse)
        mae_full_currency_denorm.append(mae)
        print(f"Full {name} - RMSE: {rmse:.4f}, MAE: {mae:.4f}")

if y_test_pre_denorm.size:
    for i, name in enumerate(currencies):
        rmse = np.sqrt(mean_squared_error(y_test_pre_denorm[:, i], y_pred_pre_denorm[:, i]))
        mae = mean_absolute_error(y_test_pre_denorm[:, i], y_pred_pre_denorm[:, i])
        rmse_pre_list_denorm.append(rmse)
        mae_pre_list_denorm.append(mae)
        print(f"Pre {name} - RMSE: {rmse:.4f}, MAE: {mae:.4f}")

if y_test_post_denorm.size:
    for i, name in enumerate(currencies):
        rmse = np.sqrt(mean_squared_error(y_test_post_denorm[:, i], y_pred_post_denorm[:, i]))
        mae = mean_absolute_error(y_test_post_denorm[:, i], y_pred_post_denorm[:, i])
        rmse_post_list_denorm.append(rmse)
        mae_post_list_denorm.append(mae)
        print(f"Post {name} - RMSE: {rmse:.4f}, MAE: {mae:.4f}")

# Save denormalized metrics (if computed)
try:
    denorm_df_rows = []
    if rmse_full_currency_denorm:
        denorm_df_rows.append({'Period':'Full', **{f'RMSE_{c}': rmse_full_currency_denorm[i] for i,c in enumerate(currencies)}, **{f'MAE_{c}': mae_full_currency_denorm[i] for i,c in enumerate(currencies)}})
    if rmse_pre_list_denorm:
        denorm_df_rows.append({'Period':'Pre-Subsidy', **{f'RMSE_{c}': rmse_pre_list_denorm[i] for i,c in enumerate(currencies)}, **{f'MAE_{c}': mae_pre_list_denorm[i] for i,c in enumerate(currencies)}})
    if rmse_post_list_denorm:
        denorm_df_rows.append({'Period':'Post-Subsidy', **{f'RMSE_{c}': rmse_post_list_denorm[i] for i,c in enumerate(currencies)}, **{f'MAE_{c}': mae_post_list_denorm[i] for i,c in enumerate(currencies)}})
    if denorm_df_rows:
        denorm_metrics_df = pd.DataFrame(denorm_df_rows)
        denorm_metrics_df.to_csv('lstm_denormalized_metrics_summary.csv', index=False)
        print("Saved lstm_denormalized_metrics_summary.csv")
except Exception as e:
    print("Could not save denormalized metrics CSV:", e)

# -------------------------
# 10. Plotting - align dates and plot Actual vs Predicted (denorm if available, else normalized)
# -------------------------
def get_sequence_dates(dataframe, n_steps):
    if not isinstance(dataframe.index, pd.DatetimeIndex):
        dataframe.index = pd.to_datetime(dataframe.index, errors='coerce')
        dataframe.dropna(axis=0, inplace=True)
    if len(dataframe) <= n_steps:
        return pd.Index([])
    return dataframe.index[n_steps:]

dates_full_seq = get_sequence_dates(df_full, N_TIMESTEPS)
dates_pre_seq = get_sequence_dates(df_pre, N_TIMESTEPS)
dates_post_seq = get_sequence_dates(df_post, N_TIMESTEPS)

dates_test_full = dates_full_seq[-len(y_test_full):] if isinstance(y_test_full, np.ndarray) and y_test_full.size and len(dates_full_seq) >= len(y_test_full) else pd.Index([])
dates_test_pre = dates_pre_seq[-len(y_test_pre):] if isinstance(y_test_pre, np.ndarray) and y_test_pre.size and len(dates_pre_seq) >= len(y_test_pre) else pd.Index([])
dates_test_post = dates_post_seq[-len(y_test_post):] if isinstance(y_test_post, np.ndarray) and y_test_post.size and len(dates_post_seq) >= len(y_test_post) else pd.Index([])

# plotting preferences: Actual = blue, Predicted = orange
actual_color = 'blue'
pred_color = 'orange'

# Helper to build DataFrame for plotting (choose denorm if available)
def build_plot_df(dates, y_test_arr, y_pred_arr, denorm_test, denorm_pred):
    if dates.empty or y_test_arr is None or y_pred_arr is None or y_test_arr.size == 0:
        return None
    if denorm_test is not None and denorm_test.size:
        df_act = pd.DataFrame(denorm_test, index=dates, columns=currencies)
        df_pred = pd.DataFrame(denorm_pred, index=dates, columns=currencies)
    else:
        df_act = pd.DataFrame(y_test_arr, index=dates, columns=currencies)
        df_pred = pd.DataFrame(y_pred_arr, index=dates, columns=currencies)
    return df_act, df_pred

# Pre plot
if not dates_test_pre.empty:
    plot_pair = build_plot_df(dates_test_pre, y_test_pre, y_pred_pre, y_test_pre_denorm, y_pred_pre_denorm)
    if plot_pair is not None:
        df_act_pre, df_pred_pre = plot_pair
        plt.figure(figsize=(12, 12))
        for i, col in enumerate(currencies):
            plt.subplot(3, 1, i+1)
            plt.plot(df_act_pre.index, df_act_pre[col], label='Actual (Pre)', color=actual_color)
            plt.plot(df_pred_pre.index, df_pred_pre[col], label='Predicted (Pre)', color=pred_color)
            plt.title(f'LSTM Forecast — Pre-Subsidy ({col}/NGN)')
            plt.xlabel('Date')
            plt.ylabel('Exchange Rate' + (' (₦)' if y_test_pre_denorm.size else ' (Normalized)'))
            plt.legend()
            plt.grid(True)
        plt.tight_layout()
        plt.savefig('lstm_pre_subsidy_actual_vs_predicted.png', dpi=300, bbox_inches='tight')
        plt.show()

# Post plot
if not dates_test_post.empty:
    plot_pair = build_plot_df(dates_test_post, y_test_post, y_pred_post, y_test_post_denorm, y_pred_post_denorm)
    if plot_pair is not None:
        df_act_post, df_pred_post = plot_pair
        plt.figure(figsize=(12, 12))
        for i, col in enumerate(currencies):
            plt.subplot(3, 1, i+1)
            plt.plot(df_act_post.index, df_act_post[col], label='Actual (Post)', color=actual_color)
            plt.plot(df_pred_post.index, df_pred_post[col], label='Predicted (Post)', color=pred_color)
            plt.title(f'LSTM Forecast — Post-Subsidy ({col}/NGN)')
            plt.xlabel('Date')
            plt.ylabel('Exchange Rate' + (' (₦)' if y_test_post_denorm.size else ' (Normalized)'))
            plt.legend()
            plt.grid(True)
        plt.tight_layout()
        plt.savefig('lstm_post_subsidy_actual_vs_predicted.png', dpi=300, bbox_inches='tight')
        plt.show()

# Combined pre+post (actuals) with combined predicted series
if (not dates_test_pre.empty) and (not dates_test_post.empty):
    # Build frames (use denorm if available)
    pre_pair = build_plot_df(dates_test_pre, y_test_pre, y_pred_pre, y_test_pre_denorm, y_pred_pre_denorm)
    post_pair = build_plot_df(dates_test_post, y_test_post, y_pred_post, y_test_post_denorm, y_pred_post_denorm)
    if pre_pair is not None and post_pair is not None:
        df_pre_act, df_pre_pred = pre_pair
        df_post_act, df_post_pred = post_pair
        plt.figure(figsize=(12, 12))
        for i, col in enumerate(currencies):
            plt.subplot(3, 1, i+1)
            plt.plot(df_pre_act.index, df_pre_act[col], label='Actual (Pre)', color='blue')
            plt.plot(df_post_act.index, df_post_act[col], label='Actual (Post)', color='darkblue')
            combined_pred = pd.concat([df_pre_pred[col], df_post_pred[col]]).sort_index()
            plt.plot(combined_pred.index, combined_pred.values, label='Predicted (Combined)', color='orange', linestyle='--')
            plt.axvline(SUBSIDY_REMOVAL_DATE, color='gray', linestyle=':', label='Subsidy Removal Date')
            plt.title(f'Actual vs Predicted — Pre & Post ({col}/NGN)')
            plt.xlabel('Date')
            plt.ylabel('Exchange Rate' + (' (₦)' if (y_test_pre_denorm.size or y_test_post_denorm.size) else ' (Normalized)'))
            plt.legend()
            plt.grid(True)
            plt.xticks(rotation=30)
        plt.tight_layout()
        plt.savefig('lstm_combined_pre_post_actual_vs_predicted.png', dpi=300, bbox_inches='tight')
        plt.show()

# -------------------------
# 11. Bar charts for normalized and denormalized RMSE/MAE
# -------------------------
# Normalized metrics (from earlier metrics_summary)
try:
    rmse_pre_list = metrics_summary['Pre-Subsidy']['RMSE_Currency']
    mae_pre_list = metrics_summary['Pre-Subsidy']['MAE_Currency']
    rmse_post_list = metrics_summary['Post-Subsidy']['RMSE_Currency']
    mae_post_list = metrics_summary['Post-Subsidy']['MAE_Currency']
    if rmse_pre_list and rmse_post_list and mae_pre_list and mae_post_list:
        x = np.arange(len(currencies))
        width = 0.35
        fig, axes = plt.subplots(1, 2, figsize=(14,6))
        # RMSE normalized
        axes[0].bar(x - width/2, rmse_pre_list, width, label='Pre-Subsidy')
        axes[0].bar(x + width/2, rmse_post_list, width, label='Post-Subsidy')
        axes[0].set_xticks(x); axes[0].set_xticklabels(currencies)
        axes[0].set_title('Normalized RMSE: Pre vs Post')
        axes[0].legend(); axes[0].grid(axis='y', linestyle='--', alpha=0.6)
        # annotate
        for rect in axes[0].patches: pass
        # MAE normalized
        axes[1].bar(x - width/2, mae_pre_list, width, label='Pre-Subsidy')
        axes[1].bar(x + width/2, mae_post_list, width, label='Post-Subsidy')
        axes[1].set_xticks(x); axes[1].set_xticklabels(currencies)
        axes[1].set_title('Normalized MAE: Pre vs Post')
        axes[1].legend(); axes[1].grid(axis='y', linestyle='--', alpha=0.6)
        plt.tight_layout()
        plt.savefig('lstm_normalized_rmse_mae_bar.png', dpi=300, bbox_inches='tight')
        plt.show()
    else:
        print("Skipping normalized RMSE/MAE bar chart (insufficient normalized metrics).")
except Exception as e:
    print("Skipping normalized RMSE/MAE bar chart due to error:", e)

# Denormalized metrics (if computed)
try:
    if rmse_pre_list_denorm and rmse_post_list_denorm and mae_pre_list_denorm and mae_post_list_denorm:
        x = np.arange(len(currencies))
        width = 0.35
        fig, axes = plt.subplots(1, 2, figsize=(14,6))
        axes[0].bar(x - width/2, rmse_pre_list_denorm, width, label='Pre-Subsidy')
        axes[0].bar(x + width/2, rmse_post_list_denorm, width, label='Post-Subsidy')
        axes[0].set_xticks(x); axes[0].set_xticklabels(currencies)
        axes[0].set_title('Denormalized RMSE: Pre vs Post')
        axes[0].legend(); axes[0].grid(axis='y', linestyle='--', alpha=0.6)
        axes[1].bar(x - width/2, mae_pre_list_denorm, width, label='Pre-Subsidy')
        axes[1].bar(x + width/2, mae_post_list_denorm, width, label='Post-Subsidy')
        axes[1].set_xticks(x); axes[1].set_xticklabels(currencies)
        axes[1].set_title('Denormalized MAE: Pre vs Post')
        axes[1].legend(); axes[1].grid(axis='y', linestyle='--', alpha=0.6)
        plt.tight_layout()
        plt.savefig('lstm_denormalized_rmse_mae_bar.png', dpi=300, bbox_inches='tight')
        plt.show()
    else:
        print("Skipping denormalized RMSE/MAE bar chart (insufficient denormalized metrics).")
except Exception as e:
    print("Skipping denormalized RMSE/MAE bar chart due to error:", e)

# -------------------------
# 12. Final: print & save summary
# -------------------------
print("\nFinal metrics summary (normalized):")
display(norm_metrics_df)

if 'denorm_metrics_df' in locals():
    print("\nFinal metrics summary (denormalized):")
    display(denorm_metrics_df)

print("\nAll done. Figures saved to current working directory.")
